<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/Brent_Kung_Scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zncof1k3
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zncof1k3
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=7ab346368e4feff177816a61496f253c0a738a8d22470ec04c64e970665b46d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-dix1x8m7/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
# Implements Brent-Kung (BK) paralle scan.
# Unlike Kogge-Stone (KS) BK uses a reduction tree inorder to generate N sums in
# logN time.
# The first phase computes N-1 total computations. Setting up intermediary 
# values for the second phase of computation.
# The second phase uses a reverse tree inorder to distrupte the intermediary values,
# to terms that still require futher additions.
# The secret to this algorithm is how ittakes advantage of more a much complex 
# indexing, inorder to minimize the overall computations required.
#
# 1st for loop in the kernel : 
#         uses the indexing scheme: (threadIdx.x + 1) * 2 * stride - 1, at 
#         the begining of each loop inorder to reduce to ensure the correct 
#         intermediary values are summed by each thread within a block. The only 
#         required check being to ensure that no thread is indexing outside of  
#         its alloted section.(n-1) total operations

# 2nd for loop in kernel:
#         implements reverse tree to prudce final sum calcualtions.Using the 
#         the same indexing scheme as in the first loop. (N-1 logN)
#      
# Total Operations: 2N-2logN 
%%cu
#include <iostream>
#define SECTION_SIZE 8


__global__ void brent_kung_scan_kernel(float* input,float* output,int input_size);
void print_array(float *array, int width);

int main(){
    
    float host_input[] = {3,1,7,0,4,1,6,3};
    float host_output[SECTION_SIZE];
    float *device_input, *device_output;

    size_t mem_size = SECTION_SIZE * sizeof(float);

    cudaMalloc((void **) &device_input,mem_size);
    cudaMalloc((void **) &device_output,mem_size);

    cudaMemcpy(device_input,host_input,mem_size,cudaMemcpyHostToDevice);
    
    dim3 dim_block(32);
    dim3 dim_grid(32);

    brent_kung_scan_kernel<<<dim_grid,dim_block>>>(device_input,device_output,SECTION_SIZE);

    cudaMemcpy(host_output,device_output,mem_size,cudaMemcpyDeviceToHost);

    printf("input:");
    print_array(host_input,SECTION_SIZE);
    
    printf("\n output:");
    print_array(host_output,SECTION_SIZE);
    
    cudaFree(device_input);
    cudaFree(device_output);
    return 0;
}

void print_array(float *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}


__global__ void brent_kung_scan_kernel(float* input,float* output, int input_size)
{
    __shared__ float ans[SECTION_SIZE];
    int i = 2 * blockIdx.x * blockDim.x + threadIdx.x;
    int index =0;
 
    if (i < input_size)
       ans[threadIdx.x] = input[i];
 
   if ( i + blockDim.x < input_size)
       ans[threadIdx.x + blockDim.x] = input[i + blockDim.x];
 
   for (unsigned int stride = 1; stride <= blockDim.x; stride *= 2){
       __syncthreads();
      index = (threadIdx.x + 1) * 2 * stride - 1;
       if(index < SECTION_SIZE)
           ans[index] += ans[index - stride];
   }
 
   for (int stride = SECTION_SIZE/4; stride > 0; stride /= 2){
       __syncthreads();
       index = (threadIdx.x + 1) * stride * 2 - 1;
       if ( index + stride < SECTION_SIZE)
          ans[index + stride] += ans[index];
   }

    __syncthreads();

    if (i < input_size)
       output[i] = ans[threadIdx.x];
 
    if (i + blockDim.x < input_size)
       output[ i + blockDim.x] = ans[threadIdx.x + blockDim.x];  
   }


input:3	1	7	0	4	1	6	3	

 output:3	4	11	11	15	16	22	25	

